In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import numpy as np
import pandas as pd

In [4]:
from drillbit import BitcoinEnvironmentUtility
from drillbit.__new_objects import Rig as RigManager, Product as ProductManager, \
    Cooling as CoolingManager, Project as ProjectManager, \
    OverClock, RigOperator
from drillbit.__new_units__ import MagnitudeTable, Hashes, HashRate, Time, Power, Energy, Efficiency, EnergyPrice


from environment.serializers import EnvironmentSerializer as EnviroSer, BlockScheduleSerializer as BSchedSer, \
    BitcoinPriceSerializer as BPSer, TransactionFeesSerializer as FeeSer, HashRateSerializer as HRSer
from environment.models import Environment, BlockSchedule, BitcoinPrice, TransactionFees

from products.models import Rig, Cooling, HeatRejection, Electrical, RejectionCurve
from products.serializers import RigSerializer, CoolingSerializer, HeatRejectionSerializer, ElectricalSerializer

from projects.models import RigForProject, InfraForProject, Project, Projects
from projects.serializers import RigForProjectSerializer, InfraForProjectSerializer, ProjectSerializer, ProjectsSerializer, \
    ProjectScalingSerializer, ProjectCostsSerializer

In [5]:
import numpy as np
import pandas as pd
import numpy_financial as npf

import finstat as fs

from drillbit.statements.funcs import total_energy, win_percentage
# , expected_difficulty, hashes_to_hash_rate, win_percentage, miner_amort, amort_sched

In [6]:
from drillbit.statements.statements import init_environment

In [25]:
rej = HeatRejection.objects.latest('created_at')

In [27]:
rej.curve.a, rej.curve.b

(-66381.81818181818, 7326181.818181817)

In [7]:
btc = BitcoinEnvironmentUtility()

env = Environment.objects.latest('created_at')
blocks = BSchedSer(env.block_schedule, bitcoin_utility=btc)
price = BPSer(env.bitcoin_price, bitcoin_utility=btc)
fees = FeeSer(env.transaction_fees, bitcoin_utility=btc)
hash_rate = HRSer(env.hash_rate, bitcoin_utility=btc)

In [8]:
env = init_environment(
    blocks.to_schedule(),
    price.to_schedule().forecast,
    fees.to_schedule().forecast,
    hash_rate.to_schedule().forecast * 1e6 * 1e12 # convert from M TH/s to H/s
)

In [9]:
rig = Rig.objects.get(make='Antminer', model='S19', generation=None)
rig = rig.as_drillbit_object()

In [10]:
rigops = RigForProject.objects.latest('created_at')

In [11]:
rig = rigops.as_drillbit_object()

In [12]:
project = rigops.project
tax_rate = .35

In [13]:
project = Project.objects.latest('created_at')

In [14]:
project = project.as_drillbit_object()

In [15]:
block_schedule = blocks.to_schedule()
n = block_schedule.shape[0]

In [30]:
energy_price = np.repeat(project.energy_price, n)
opex = np.repeat(0, n)
property_taxes = np.repeat(0, n)
pue = project.project_pue
number_of_rigs = np.repeat(project.rigs.schedule, n)

In [31]:
power_per_rig = project.rigs.OC.power_by_factor().consumption_per_block()
power_per_rig_in_joules = project.rigs.OC.power_by_factor().consumption_per_block().in_joules()

In [32]:
hash_rate = project.rigs.OC.hash_rate_by_factor().raw

In [33]:
hashes = project.rigs.OC.hash_rate_by_factor().hashes_per_block().raw

In [38]:
stat = fs.FinancialStatement(name=project.name, periods=block_schedule.period)
stat.add_factor('tax_rate', tax_rate)
stat.add_statement(name='Environment', short_name='env')
stat.add_statement(name='Income Statement')


stat.env.add_account(number_of_rigs, name='Number of Miners', short_name='n_miners')
stat.env.add_account(
    fs.arr.multiply(stat.n_miners, power_per_rig_in_joules), 
    name='Energy (J) - Miner', 
    short_name='miner_energy_in_joules',
    hide=True
)
stat.env.add_account(
    fs.arr.multiply(stat.n_miners, power_per_rig), 
    name='Energy - Miner', 
    short_name='miner_energy'
)

stat.env.add_account(total_energy(stat.miner_energy, pue), name='Energy - Infra', short_name='infra_energy')
stat.env.add_account(fs.arr.add(stat.infra_energy, stat.miner_energy), name='Energy')

stat.env.add_account(fs.arr.multiply(hash_rate, stat.n_miners), name='Hash Rate')
stat.env.add_account(fs.arr.multiply(hashes, stat.n_miners), name='Hashes')

stat.env.add_account(win_percentage(stat.hashes, env.difficulty), name='Win %', short_name='win_per')

stat.env.add_account(fs.arr.multiply(stat.win_per, env.reward), name='BTC Reward', short_name='btc_reward')
stat.env.add_account(fs.arr.multiply(stat.win_per, env.fees), name='Transaction Fees', short_name='traxn_fees')
# stat.env.add_account(fs.arr.multiply(stat.btc_reward, mine.pool_fee), name='Pool Fees (\u0243)', short_name='pool_fees_in_btc')
# stat.env.add_account(fs.arr.add(stat.btc_reward, stat.traxn_fees, -stat.pool_fees_in_btc), name='BTC Mined', short_name='btc_mined')

stat.istat.add_account(fs.arr.multiply(stat.btc_reward, env.btc_price), name='Revenue - Reward', short_name='reward_rev')
# stat.istat.add_account(fs.arr.multiply(stat.traxn_fees, env.btc_price), name='Revenue - Fees', short_name='fee_rev')
# stat.istat.add_account(fs.arr.add(stat.fee_rev, stat.reward_rev), name='Gross Revenue', short_name='gross_rev')

# stat.istat.add_account(fs.arr.multiply(stat.pool_fees_in_btc, env.btc_price), name='Pool Fees', short_name='pool_fees')
# stat.istat.add_account(fs.arr.add(stat.gross_rev, -stat.pool_fees), name='Net Revenue', short_name='net_rev')
# stat.istat.add_account(fs.arr.multiply(stat.btc_mined, env.btc_price), name='Test Net Revenue', short_name='test_net_rev', hide=True)

stat.istat.add_account(fs.arr.multiply(stat.energy, energy_price), name='Energy Expenses', short_name='energy_exp')
stat.istat.add_account(fs.arr.add(stat.reward_rev, -stat.energy_exp), name='Gross Profit')
# stat.istat.add_account(fs.arr.divide(stat.gp, stat.net_rev), name='Gross Margin', hide=True)

stat.istat.add_account(opex, name='Operations', short_name='ops')
stat.istat.add_account(property_taxes, name='Property Taxes', short_name='prop_tax')

stat.istat.add_account(fs.arr.add(stat.gp, -stat.ops, -stat.prop_tax), name='EBITDA')

stat.istat.add_account(miner_amort(mine, periods=stat.periods), name='Miner Amortization', short_name='miner_amort')
stat.istat.add_account(amort_sched(mine.build_cost, 1, mine.property_amort, mine.implement.start_in_blocks(), periods=stat.periods), name='Building Amortization', short_name='build_amort')
stat.istat.add_account(amort_sched(mine.cost_of_cooling, 1, mine.cooling.amort, mine.implement.start_in_blocks(), periods=stat.periods), name='Cooling Amortization', short_name='cool_amort')

stat.istat.add_account(fs.arr.add(stat.miner_amort, stat.build_amort, stat.cool_amort), name='Depreciation for Taxes', short_name='tax_depn')

stat.istat.add_account(fs.arr.add(stat.ebitda, -stat.tax_depn), name='EBIT')
stat.istat.add_account(fs.arr.multiply(stat.ebit, stat.tax_rate), name='Taxes')

stat.istat.add_account(fs.arr.add(stat.ebit, -stat.taxes), name='Profit, if sold', short_name='profit_sold')
stat.istat.add_account(fs.arr.add(stat.ebitda, -stat.taxes), name='Operating Cash Flow, if sold', short_name='op_flow_sold')

stat.istat.add_account(fs.arr.add(stat.energy_exp, stat.ops, stat.prop_tax, stat.taxes), name='Cash Expenses', short_name='cash_exp', hide=True)
stat.istat.add_account(fs.arr.divide(stat.cash_exp, env.btc_price), name='BTC Converted for Expenses', short_name='converted')

stat.istat.add_account(fs.arr.add(stat.btc_mined, -stat.converted), name='BTC Earned', short_name='btc_earned')
stat.istat.add_account(fs.arr.cumsum(stat.btc_earned), name='BTC, if held', short_name='btc_held')

stat.istat.add_account(fs.arr.multiply(stat.btc_held, env.btc_price), name='BTC Value, if held', short_name='btc_value_held')


In [35]:
env

period,2023-02-14 11:48,2023-02-14 11:58,2023-02-14 12:08,2023-02-14 12:18,2023-02-14 12:28,2023-02-14 12:38,2023-02-14 12:48,2023-02-14 12:58,2023-02-14 13:08,2023-02-14 13:18,...,2025-07-22 07:28,2025-07-22 07:38,2025-07-22 07:48,2025-07-22 07:58,2025-07-22 08:08,2025-07-22 08:18,2025-07-22 08:28,2025-07-22 08:38,2025-07-22 08:48,2025-07-22 08:58
Item,,,,,,,,,,,,,,,,,,,,,
Block ID,712000,712001,712002,712003,712004,712005,712006,712007,712008,712009,...,839990,839991,839992,839993,839994,839995,839996,839997,839998,839999
Block Reward,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,...,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25
BTC Price,19991.075731,19976.418336,19984.097595,19990.109478,19981.127843,19965.279216,19963.001474,19957.69741,19977.050085,19961.872918,...,34716.791772,34715.699882,34724.299618,34690.239508,34698.816423,34729.963617,34719.697698,34714.764735,34736.865023,34758.435753
Transaction Fees,0.069457,0.069475,0.069554,0.069561,0.069583,0.069618,0.069607,0.069599,0.069536,0.069473,...,0.080942,0.080945,0.080896,0.080846,0.080844,0.080745,0.080724,0.080754,0.080789,0.080746
Network Hash Rate,225174937071199977472.0,225198066899999981568.0,225399199611500003328.0,225287872282900004864.0,225272971227499986944.0,225399820644300029952.0,225459539720499986432.0,225478836056400003072.0,225405338215400013824.0,225474762787600007168.0,...,352532701803899977728.0,352816627969100021760.0,353041940490199957504.0,352967278497200013312.0,353089057489500045312.0,352782279612699967488.0,352650274379099996160.0,352660990812799959040.0,352563399427099983872.0,352488621173800042496.0
Market Rewards,124944.223321,124852.614602,124900.609967,124938.184239,124882.049019,124782.995102,124768.75921,124735.60881,124856.563029,124761.705739,...,216979.948577,216973.124263,217026.872615,216813.996924,216867.602642,217062.272608,216998.110613,216967.279591,217105.406392,217240.223458
Market Fees,1388.514973,1387.858482,1389.973798,1390.528589,1390.353103,1389.943262,1389.561298,1389.035726,1389.118671,1386.807971,...,2810.063644,2810.05637,2809.043382,2804.561057,2805.18556,2804.28278,2802.719779,2803.357059,2806.366888,2806.614396
Market Revenue,126332.738295,126240.473083,126290.583765,126328.712828,126272.402122,126172.938364,126158.320508,126124.644536,126245.6817,126148.513711,...,219790.012221,219783.180633,219835.915997,219618.55798,219672.788201,219866.555387,219800.830392,219770.63665,219911.77328,220046.837854
Network Hashes,135104962242719994871808.0,135118840139999982125056.0,135239519766899995967488.0,135172723369740007899136.0,135163782736499995312128.0,135239892386580017446912.0,135275723832299995791360.0,135287301633840001318912.0,135243202929240016158720.0,135284857672560004825088.0,...,211519621082339973529600.0,211689976781460025114624.0,211825164294119959298048.0,211780367098320003268608.0,211853434493700038197248.0,211669367767619996745728.0,211590164627460001366016.0,211596594487679959171072.0,211538039656259992420352.0,211493172704280032837632.0


In [36]:
stat.env

period,2023-02-14 11:48,2023-02-14 11:58,2023-02-14 12:08,2023-02-14 12:18,2023-02-14 12:28,2023-02-14 12:38,2023-02-14 12:48,2023-02-14 12:58,2023-02-14 13:08,2023-02-14 13:18,...,2025-07-22 07:28,2025-07-22 07:38,2025-07-22 07:48,2025-07-22 07:58,2025-07-22 08:08,2025-07-22 08:18,2025-07-22 08:28,2025-07-22 08:38,2025-07-22 08:48,2025-07-22 08:58
Item,,,,,,,,,,,,,,,,,,,,,
Number of Miners,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,...,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346,7531.85346
Energy - Miner,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,...,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024,5893424.271024
Energy - Infra,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,...,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974,325611.690974
Energy,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,...,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998,6219035.961998
Hash Rate,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,...,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0,1552541173196167936.0
Hashes,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,...,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0,931524703917700808704.0
Win %,0.006895,0.006894,0.006888,0.006891,0.006892,0.006888,0.006886,0.006886,0.006888,0.006886,...,0.004404,0.0044,0.004398,0.004399,0.004397,0.004401,0.004403,0.004402,0.004404,0.004405
BTC Reward,0.043093,0.043089,0.04305,0.043072,0.043075,0.04305,0.043039,0.043035,0.043049,0.043036,...,0.027525,0.027503,0.027485,0.027491,0.027482,0.027506,0.027516,0.027515,0.027523,0.027529
Transaction Fees,0.000479,0.000479,0.000479,0.000479,0.00048,0.00048,0.000479,0.000479,0.000479,0.000478,...,0.000356,0.000356,0.000356,0.000356,0.000355,0.000355,0.000355,0.000356,0.000356,0.000356


In [21]:
stat.istat

period,2023-02-14 11:48,2023-02-14 11:58,2023-02-14 12:08,2023-02-14 12:18,2023-02-14 12:28,2023-02-14 12:38,2023-02-14 12:48,2023-02-14 12:58,2023-02-14 13:08,2023-02-14 13:18,...,2025-07-22 07:28,2025-07-22 07:38,2025-07-22 07:48,2025-07-22 07:58,2025-07-22 08:08,2025-07-22 08:18,2025-07-22 08:28,2025-07-22 08:38,2025-07-22 08:48,2025-07-22 08:58
Item,,,,,,,,,,,,,,,,,,,,,
Revenue - Reward,861.48136,860.761308,860.323811,861.007887,860.67796,859.511304,859.185608,858.883817,859.996991,859.079031,...,955.586368,954.787338,954.414358,953.679885,953.586674,955.272634,955.347738,955.182976,956.055638,956.852272
Energy Expenses,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,...,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798,310.951798
Gross Profit,550.529562,549.80951,549.372013,550.056089,549.726162,548.559506,548.233809,547.932019,549.045193,548.127232,...,644.63457,643.83554,643.46256,642.728087,642.634876,644.320836,644.39594,644.231178,645.103839,645.900473
